### inference 방법들 중 normal, self-consistency, 재추론 방법의 accuracy 비교

In [87]:
import argparse
import yaml
from pathlib import Path
from typing import Dict, Any
import re
from tqdm import tqdm 
from collections import Counter
from sklearn.model_selection import train_test_split
from typing import List

import pandas as pd
import torch
from transformers import AutoTokenizer

import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '../..')) 
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data.preprocessor import parse_problems_column, add_choices_len
from src.prompt.prompt_builder import PromptBuilder, PromptConfig
from src.training.model_loader import ModelConfig, load_model_inference

In [2]:
def create_configs(cfg_dict: Dict[str, Any]) -> tuple:
    model_cfg_dict = cfg_dict["model"].copy()
    model_cfg_dict["use_gradient_checkpointing"] = False
    model_cfg = ModelConfig(**model_cfg_dict)
    
    prompt_dict = cfg_dict["inference"]["prompt"]
    prompt_cfg = PromptConfig(
        policy=prompt_dict["policy"],
        mode="test",
        verbose=False
    )
    
    inference_cfg = cfg_dict.get("inference", {})
    
    return model_cfg, prompt_cfg, inference_cfg

In [3]:
with open("../../config.yaml", "r") as f:
        cfg_dict = yaml.safe_load(f)

In [4]:
model_cfg, prompt_cfg, inference_cfg = create_configs(cfg_dict)

### Validation data 재구성

In [5]:
data_path = '../../data/train.csv'

print(f"\nLoading data from {data_path}...")
df = pd.read_csv(data_path)
df = parse_problems_column(df)
df = add_choices_len(df)
print(f"Loaded {len(df)} rows")


Loading data from ../../data/train.csv...
Loaded 2031 rows


In [6]:
valid_ratio = cfg_dict["data"]["valid_ratio"]
seed = cfg_dict["data"]["seed"]

print(f"\nSplitting data (valid_ratio={valid_ratio}, seed={seed})...")
train_df, valid_df = train_test_split(
    df,
    test_size=valid_ratio,
    stratify=df["choices_len"],
    random_state=seed,
)
print(f"Train: {len(train_df)} rows")
print(f"Valid: {len(valid_df)} rows")


Splitting data (valid_ratio=0.1, seed=42)...
Train: 1827 rows
Valid: 204 rows


In [58]:
lsat_df = pd.read_csv("./review_autosave.csv")
lsat_df = lsat_df[lsat_df['keep'] == True]

print(f"lsat_df 데이터 {len(lsat_df)}개 준비 완료")

lsat_df 데이터 270개 준비 완료


In [59]:
import ast

# 1. 'choices' 컬럼의 문자열을 리스트 객체로 일괄 변환
# 혹시 모를 에러(이미 리스트인 경우 등)를 방지하기 위해 간단한 조건문을 추가합니다.
lsat_df['choices'] = lsat_df['choices'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x
)

In [60]:
lsat_df['choices_len'] = lsat_df['choices'].apply(len)

In [61]:
lsat_df = lsat_df.drop(['group_id', 'keep'], axis=1)

In [62]:
eval_df = pd.concat([valid_df, lsat_df], axis=0, ignore_index=True)

In [63]:
len(eval_df)

474

In [66]:
eval_df['choices_len'].value_counts()

choices_len
5    394
4     80
Name: count, dtype: int64

In [75]:
prompt_cfg = PromptConfig(
        policy=cfg_dict["prompt"]["policy"],
        mode="test",
        verbose=False
    )

In [76]:
builder = PromptBuilder(prompt_cfg)
print("PromptBuilder ready!")


PromptBuilder ready!


In [72]:
def extract_answer(text: str, k: int) -> str:
    numbers = re.findall(rf'[1-{k}]', str(text))
    return numbers[-1] if numbers else "no"

### 모델 로드

In [69]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}\n")

Device: cuda



In [67]:
adapter_path = "../../outputs/reading/final_model"

print(f"Loading model from {adapter_path}...")
model = load_model_inference(model_cfg, "../../models/qwen3_14B_eng/final_model_from_serverA")
model.eval()
print("Model loaded successfully!\n")

Loading model from ../../outputs/reading/final_model...
Loading Base Model for Inference: Qwen/Qwen3-14B


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loading LoRA Adapter from: ../../models/qwen3_14B_eng/final_model_from_serverA
Model loaded successfully!



In [68]:
print(f"Loading tokenizer from {model_cfg.model_name_or_path}...")
tokenizer = AutoTokenizer.from_pretrained(
    model_cfg.model_name_or_path,
    trust_remote_code=model_cfg.trust_remote_code,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token   

Loading tokenizer from Qwen/Qwen3-14B...


## Test 시작

In [73]:
max_new_tokens = 30

In [78]:
def digit_only_probs_and_margin(step_logits: torch.Tensor, tokenizer, k: int) -> Dict[str, Any]:
    digit_tokens = [str(i) for i in range(1, k + 1)]
    digit_token_ids = []

    for digit in digit_tokens:
        encoded = tokenizer.encode(digit, add_special_tokens=False)
        if len(encoded) == 1:
            digit_token_ids.append(encoded[0])
        else:
            digit_token_ids.append(encoded[0])

    digit_logits = torch.tensor([step_logits[tid].item() for tid in digit_token_ids])
    digit_probs = torch.softmax(digit_logits, dim=-1)

    top2_values, top2_indices = torch.topk(digit_probs, k=min(2, k))

    digit_top1 = str(top2_indices[0].item() + 1)  # 1-indexed
    digit_top2 = str(top2_indices[1].item() + 1) if k >= 2 else "N/A"

    if k >= 2:
        digit_margin = (top2_values[0] - top2_values[1]).item()
    else:
        digit_margin = 0.0

    return {
        "digit_probs": digit_probs.tolist(),
        "digit_margin": digit_margin,
        "digit_top1": digit_top1,
        "digit_top2": digit_top2,
    }

In [68]:
def generate_for_row_with_retry(
    row_dict: Dict[str, Any],
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str,
    generated_text: str,
    max_new_tokens: int = 30,
) -> Dict[str, Any]:
    """
    첫 번째 예측의 확률이 낮을 때, 재고려를 유도하는 프롬프트를 추가하여 재생성
    """
    output = builder.build_message(row_dict)
    messages = output["messages"]
    
    # 재시도 프롬프트 추가
    retry_assistant = {
        "role": "assistant",
        "content": generated_text
    }

    retry_message = {
        "role": "user",
        "content": "다시 한번 신중하게 생각해서 답변해주세요. 다른 접근 방식으로 다시 풀어보세요."
    }
    messages.append(retry_assistant)
    messages.append(retry_message)
    
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(device)

    k = int(row_dict["choices_len"])
    input_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,
        )

    generated_ids = outputs.sequences[0][input_len:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # 끝에서 2번째 step의 logits 사용 (답변 digit이 나오는 위치)
    step_logits = outputs.scores[-2][0]

    top5_values, top5_indices = torch.topk(step_logits, k=5)
    probs_full = torch.softmax(step_logits, dim=-1)
    top5_candidates = []
    for rank, (logit_val, token_id) in enumerate(zip(top5_values, top5_indices)):
        top5_candidates.append({
            "rank": rank + 1,
            "token_id": token_id.item(),
            "token": tokenizer.decode([token_id.item()]),
            "logit": logit_val.item(),
            "prob_full_vocab": probs_full[token_id].item(),
        })

    digit_info = digit_only_probs_and_margin(step_logits, tokenizer, k)
    digit_margin = digit_info["digit_margin"]
    digit_probs = digit_info["digit_probs"]

    predicted_answer = extract_answer(generated_text, k=k)
    gold = str(row_dict["answer"])

    return {
        "id": row_dict["id"],
        "choices_len": k,
        "answer": gold,
        "predicted_answer": predicted_answer,
        "is_correct": predicted_answer == gold,
        "generated_text": generated_text,
        "is_retry": True,  # retry 여부 표시

        "top5_candidates": top5_candidates,

        "digit_probs_1_to_k": digit_probs,  
        "digit_margin_top1_minus_top2": digit_margin,
        "digit_top1": digit_info["digit_top1"],
        "digit_top2": digit_info["digit_top2"],

        "prompt": prompt_text,
    }

In [70]:
def generate_for_row_with_top5(
    row_dict: Dict[str, Any],
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str,
    max_new_tokens: int = 30,
) -> Dict[str, Any]:
    output = builder.build_message(row_dict)
    messages = output["messages"]

    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(device)

    k = int(row_dict["choices_len"])
    input_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,
        )

    generated_ids = outputs.sequences[0][input_len:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # 끝에서 2번째 step의 logits 사용 (답변 digit이 나오는 위치)
    step_logits = outputs.scores[-2][0]

    top5_values, top5_indices = torch.topk(step_logits, k=5)
    probs_full = torch.softmax(step_logits, dim=-1)
    top5_candidates = []
    for rank, (logit_val, token_id) in enumerate(zip(top5_values, top5_indices)):
        top5_candidates.append({
            "rank": rank + 1,
            "token_id": token_id.item(),
            "token": tokenizer.decode([token_id.item()]),
            "logit": logit_val.item(),
            "prob_full_vocab": probs_full[token_id].item(),
        })

    digit_info = digit_only_probs_and_margin(step_logits, tokenizer, k)
    digit_margin = digit_info["digit_margin"]
    digit_probs = digit_info["digit_probs"]

    predicted_answer = extract_answer(generated_text, k=k)
    gold = str(row_dict["answer"])

    return {
        "id": row_dict["id"],
        "choices_len": k,
        "answer": gold,
        "predicted_answer": predicted_answer,
        "is_correct": predicted_answer == gold,
        "generated_text": generated_text,
        "is_retry": False,  # retry 여부 표시

        "top5_candidates": top5_candidates,

        "digit_probs_1_to_k": digit_probs,  
        "digit_margin_top1_minus_top2": digit_margin,
        "digit_top1": digit_info["digit_top1"],
        "digit_top2": digit_info["digit_top2"],

        "prompt": prompt_text,
    }


### 1. normal

In [71]:
def process_normal(
    df: pd.DataFrame,
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str,
    max_new_tokens: int,
    desc: str = "Processing",
) -> pd.DataFrame:
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        row_dict = row.to_dict()
        result = generate_for_row_with_top5(
            row_dict=row_dict,
            builder=builder,
            tokenizer=tokenizer,
            model=model,
            device=device,
            max_new_tokens=max_new_tokens,
        )
        results.append(result)

    return pd.DataFrame(results)

In [79]:
print("\n" + "=" * 80)
print("Running inference on VALID set")
print("=" * 80)
valid_gen_df = process_normal(
    df=eval_df,
    builder=builder,
    tokenizer=tokenizer,
    model=model,
    device=device,
    max_new_tokens=max_new_tokens,
    desc="Valid Generation",
)

valid_acc = valid_gen_df['is_correct'].mean()
print(f"\nValid Accuracy: {valid_acc:.4f} ({valid_gen_df['is_correct'].sum()}/{len(valid_gen_df)})")


Running inference on VALID set


Valid Generation: 100%|██████████| 474/474 [14:03<00:00,  1.78s/it]


Valid Accuracy: 0.8586 (407/474)


### 2. self-consistency

In [85]:
def process_sc(
    row_dict: Dict,
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str = "cuda",
    max_new_tokens: int = 100,
    num_samples: int = 5,  # Self-Consistency 샘플링 횟수
) -> Dict:

    output = builder.build_message(row_dict)
    messages = output["messages"]
    
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    
    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(device)

    k = int(row_dict["choices_len"])
    input_len = inputs["input_ids"].shape[1]
    
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            num_return_sequences=num_samples,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,
        )

    # 첫 번째 샘플의 생성 텍스트 (기존 로직 유지)
    generated_ids = output_ids.sequences[0][input_len:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # 첫 번째 샘플의 logits 분석 (기존 로직 유지)
    step_logits = output_ids.scores[-2][0]

    top5_values, top5_indices = torch.topk(step_logits, k=5)
    probs_full = torch.softmax(step_logits, dim=-1)
    top5_candidates = []
    for rank, (logit_val, token_id) in enumerate(zip(top5_values, top5_indices)):
        top5_candidates.append({
            "rank": rank + 1,
            "token_id": token_id.item(),
            "token": tokenizer.decode([token_id.item()]),
            "logit": logit_val.item(),
            "prob_full_vocab": probs_full[token_id].item(),
        })

    digit_info = digit_only_probs_and_margin(step_logits, tokenizer, k)
    digit_margin = digit_info["digit_margin"]
    digit_probs = digit_info["digit_probs"]

    predicted_answer = extract_answer(generated_text, k=k)
    gold = str(row_dict["answer"])
    
    # 생성된 모든 결과 디코딩 및 정답 추출
    all_full_texts = []
    all_extracted_answers = []
    
    for i in range(num_samples):
        seq = output_ids.sequences[i][input_len:]
        text = tokenizer.decode(seq, skip_special_tokens=True)
        all_full_texts.append(text)
        all_extracted_answers.append(extract_answer(text, k=k))
    
    # 다수결 투표 (가장 많이 나온 정답 선택)
    valid_answers = [a for a in all_extracted_answers if a is not None]
    
    if valid_answers:
        final_answer = Counter(valid_answers).most_common(1)[0][0]
    else:
        final_answer = None
    
    return {
        "id": row_dict.get("id"),
        "choices_len": k,

        "answer": gold,
        "final_answer": final_answer,  # Self-Consistency 결과
        "all_answers": all_extracted_answers,  # 디버깅용 전체 정답 목록
        "predicted_answer": predicted_answer,  # 첫 번째 샘플 답
        "is_correct": final_answer == gold,

        "generated_text": generated_text,  # 첫 번째 샘플 텍스트
        "all_generated_texts": all_full_texts,  # 모든 샘플 텍스트
        
        "top5_candidates": top5_candidates,

        "digit_probs_1_to_k": digit_probs,  
        "digit_margin_top1_minus_top2": digit_margin,
        "digit_top1": digit_info["digit_top1"],
        "digit_top2": digit_info["digit_top2"],

        "prompt": prompt_text,
    }

In [86]:
sc_result_list = []

for idx, row in tqdm(eval_df.iterrows(), total=len(eval_df), desc="Inference"):
    if (idx + 1) % 100 == 0:
        print(f"  [{idx+1}/{len(eval_df)}]")
    
    row_dict = row.to_dict()
    result = process_sc(
        row_dict,
        builder,
        tokenizer,
        model,
        device=device,
        max_new_tokens=max_new_tokens
    )
    sc_result_list.append(result)

valid_gen_sc_df = pd.DataFrame(sc_result_list)
valid_sc_acc = valid_gen_sc_df['is_correct'].mean()
print(f"\nValid Accuracy: {valid_sc_acc:.4f} ({valid_gen_sc_df['is_correct'].sum()}/{len(valid_gen_sc_df)})")


Inference:  21%|██        | 99/474 [06:59<28:17,  4.53s/it]

  [100/474]


Inference:  42%|████▏     | 199/474 [14:14<17:44,  3.87s/it]

  [200/474]


Inference:  63%|██████▎   | 299/474 [23:14<15:53,  5.45s/it]

  [300/474]


Inference:  84%|████████▍ | 399/474 [32:42<07:07,  5.69s/it]

  [400/474]


Inference: 100%|██████████| 474/474 [39:46<00:00,  5.04s/it]


Valid Accuracy: 0.8418 (399/474)


### 3. 어려운 문제 로직 적용

In [ ]:
def is_logic_type(text):
        calc_patterns = r"(\d+(\.\d+)?%|\d+배|\d+원|\d+달러|계산|합계|평균|비율|변화율)"
        logic_keywords = [
            "GDP", "CPI", "물가", "소득", "금리", "수요", "공급", "탄력성",
            "비용", "이윤", "이익", "상승", "하락", "증가", "감소", "환율",
            "명목", "실질", "인플레이션", "시장 구조", "독점", "과점", "진단", "증상"
        ]
        if re.search(calc_patterns, text) and any(keyword in text for keyword in logic_keywords): return True
        return False

In [109]:
eval_df['is_logic'] = eval_df['paragraph'].apply(is_logic_type)

# 3. 결과 집계
logic_count = eval_df['is_logic'].sum()
total_count = len(eval_df)
ratio = (logic_count / total_count) * 100

print(f"- 전체 데이터 개수: {total_count}개")
print(f"- 로직 타입으로 판정된 개수: {logic_count}개")
print(f"- 비중: {ratio:.2f}%")

# 4. 실제로 어떤 것들이 잡혔는지 샘플 확인
# display(eval_df[eval_df['is_logic']][['id', 'paragraph']].head(10))

- 전체 데이터 개수: 474개
- 로직 타입으로 판정된 개수: 253개
- 비중: 53.38%


In [88]:
SYSTEM_PROMPT ="""당신은 정확하고 객관적인 분석 능력을 갖춘 고도로 훈련된 '논리 분석가(Logic Analyst)'입니다.
당신의 임무는 주어진 [질문]과 관련 정보를 분석하여 인과 관계, 경제학적 원리, 수치 계산, 또는 임상적 증거에 기반해 최고 수준의 논리적 정확성과 명확성을 확보하고, 가장 타당한 정답을 도출하는 것입니다.
어떠한 대화형 문구나 사족도 포함하지 말고, 오직 최종 정답만을 출력하십시오.
"""

In [94]:
USER_PROMPT = """지문:
{paragraph}


질문:
{question}


선택지:
{choices}


문제를 해결할 때는 다음 단계를 **반드시 순서대로** 따르고, 각 단계별로 명확한 설명을 다음 단계에 제공하십시오:

**1. 유형 식별**: 문제의 핵심 분야(예: 경제학(계산/이론), 심리학(진단), 통계/인구학)를 명확히 식별하십시오.
**2. 핵심 정보 추출**: 주어진 [질문]과 [보기]에서 분석에 필요한 모든 핵심 변수(예: 금리, 수요, 증상, 수치 데이터)와 조건(예: 상승/하락, ~가 발생했다면)을 빠짐없이 정확하게 나열하십시오.
**3. 관련 원리 및 공식 적용**: 식별된 유형에 따라 해당 문제에 적용되는 주요 법칙, 이론, 또는 공식(예: 수요-공급 곡선의 이동, GDP 공식, 심리 진단 기준)을 제시하고 그 내용을 간략히 설명하십시오.
**4. 단계별 논리적 추론**: 추출된 조건과 적용된 원리를 바탕으로, 최종 결과에 도달하는 논리적 과정을 2~4문장으로 간결하게 서술하십시오. 복잡한 추론의 경우 핵심적인 인과 관계를 명확히 제시하십시오 (예: 금리 인상 -> 투자 감소 -> 총수요 감소).
**5. 최종 결론 및 정답 도출**: 단계적 추론 결과를 [보기] 및 [선택지]와 면밀히 비교하여, 가장 논리적으로 타당한 1, 2, 3, 4 중 하나의 숫자를 최종 정답으로 명확히 선택하십시오.

"""

In [95]:
def format_choices(choices: List[str]) -> str:
        """
        리스트 형태의 선택지를 번호와 함께 문자열로 포맷팅합니다.
        
        Args:
            choices: 선택지 텍스트들이 담긴 리스트
            
        Returns:
            "1 - 선택지" 형태로 줄바꿈된 전체 선택지 문자열
        """
        return "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(choices)])

In [ ]:
def generate_for_hard_problem(
    row_dict: Dict[str, Any],
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str,
    max_new_tokens: int = 30,
) -> Dict[str, Any]:
    choices_string = format_choices(row_dict['choices'])

    user_message = USER_PROMPT.format(
        paragraph=row_dict["paragraph"],
        question=row_dict["question"],
        choices=choices_string,
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_message},
    ]
        

    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(device)

    k = int(row_dict["choices_len"])
    input_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,
        )

    generated_ids = outputs.sequences[0][input_len:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # 끝에서 2번째 step의 logits 사용 (답변 digit이 나오는 위치)
    step_logits = outputs.scores[-2][0]

    top5_values, top5_indices = torch.topk(step_logits, k=5)
    probs_full = torch.softmax(step_logits, dim=-1)
    top5_candidates = []
    for rank, (logit_val, token_id) in enumerate(zip(top5_values, top5_indices)):
        top5_candidates.append({
            "rank": rank + 1,
            "token_id": token_id.item(),
            "token": tokenizer.decode([token_id.item()]),
            "logit": logit_val.item(),
            "prob_full_vocab": probs_full[token_id].item(),
        })

    digit_info = digit_only_probs_and_margin(step_logits, tokenizer, k)
    digit_margin = digit_info["digit_margin"]
    digit_probs = digit_info["digit_probs"]

    predicted_answer = extract_answer(generated_text, k=k)
    gold = str(row_dict["answer"])

    return {
        "id": row_dict["id"],
        "choices_len": k,
        "answer": gold,
        "predicted_answer": predicted_answer,
        "is_correct": predicted_answer == gold,
        "generated_text": generated_text,
        "is_retry": False,  # retry 여부 표시

        "top5_candidates": top5_candidates,

        "digit_probs_1_to_k": digit_probs,  
        "digit_margin_top1_minus_top2": digit_margin,
        "digit_top1": digit_info["digit_top1"],
        "digit_top2": digit_info["digit_top2"],

        "prompt": prompt_text,
    }


In [100]:
def process_hard(
    df: pd.DataFrame,
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str,
    max_new_tokens: int,
    desc: str = "Processing",
) -> pd.DataFrame:
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        row_dict = row.to_dict()
        # 어려운 문제일 때
        if row_dict['is_logic']:
            print("어려운 문제로 다른 프롬프트 적용")
            result = generate_for_hard_problem(
                row_dict=row_dict,
                builder=builder,
                tokenizer=tokenizer,
                model=model,
                device=device,
                max_new_tokens=max_new_tokens,
            )
        # 일반 문제일 때
        else:
            result = generate_for_row_with_top5(
                row_dict=row_dict,
                builder=builder,
                tokenizer=tokenizer,
                model=model,
                device=device,
                max_new_tokens=max_new_tokens,
            )
        results.append(result)

    return pd.DataFrame(results)

In [111]:
print("\n" + "=" * 80)
print("Running inference on VALID set - with hard problem logic")
print("=" * 80)
valid_gen_hard_df = process_hard(
    df=eval_df,
    builder=builder,
    tokenizer=tokenizer,
    model=model,
    device=device,
    max_new_tokens=max_new_tokens,
    desc="Valid Generation",
)

valid_hard_acc = valid_gen_hard_df['is_correct'].mean()
print(f"\nValid Accuracy: {valid_hard_acc:.4f} ({valid_gen_hard_df['is_correct'].sum()}/{len(valid_gen_hard_df)})")


Running inference on VALID set - with hard problem logic


Valid Generation:   0%|          | 0/474 [00:00<?, ?it/s]

어려운 문제로 다른 프롬프트 적용


Valid Generation:   1%|          | 3/474 [00:05<13:11,  1.68s/it]

어려운 문제로 다른 프롬프트 적용


...

어려운 문제로 다른 프롬프트 적용


Valid Generation: 100%|█████████▉| 473/474 [15:25<00:02,  2.08s/it]

어려운 문제로 다른 프롬프트 적용


Valid Generation: 100%|██████████| 474/474 [15:28<00:00,  1.96s/it]


Valid Accuracy: 0.8418 (399/474)


### 결과 확인

In [102]:
target_dfs = [
    ("Normal", valid_gen_df),
    ("Self Consistency", valid_gen_sc_df),
    ("Hard Problem", valid_gen_hard_df),
]

for name, df in target_dfs:
    acc = df['is_correct'].mean()
    correct_count = df['is_correct'].sum()
    total_coiunt = len(df)
    print(f"{name:<15} : {acc:.4f} ({correct_count}/{total_count})")

Normal          : 0.8586 (407/474)
Self Consistency : 0.8418 (399/474)
Hard Problem    : 0.8523 (404/474)


### 다른 프롬프트 적용

In [113]:
SYSTEM_PROMPT_1 = """
당신은 복잡한 문제를 구조화하고 풀이 전략을 설계하는 '문제 분석 전문가'입니다. 지문을 읽고 문제의 본질을 파악하여 논리적 해결 로직을 구성하는 것이 당신의 역할입니다.
"""

In [114]:
USER_PROMPT_1 = """지문: {paragraph}
질문: {question}
선택지: {choices}

위 문제를 해결하기 위해 다음 3가지를 분석하십시오.

1. 문제 유형 식별: 문제의 핵심 분야(경제, 한국사, 논리 등)와 성격(계산형, 사실 확인형, 추론형)을 정의하십시오.
2. 핵심 정보 추출: 지문과 질문에서 정답 도출에 반드시 필요한 변수, 수치, 인과 관계를 추출하십시오.
3. 해결 단계 설계: 정답을 도출하기 위해 거쳐야 할 논리적 단계(Step-by-step plan)를 순서대로 나열하십시오. (예: A사건의 원인 파악 -> B정책과의 관련성 분석 -> 결과 도출)
"""

In [115]:
row_dict = eval_df.to_dict()

In [116]:
choices_string = format_choices(row_dict['choices'])


In [117]:
user_message = USER_PROMPT_1.format(
        paragraph=row_dict["paragraph"],
        question=row_dict["question"],
        choices=choices_string,
    )

In [118]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT_1},
    {"role": "user", "content": user_message},
]

In [131]:

prompt_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

inputs = tokenizer(
    prompt_text,
    return_tensors="pt",
    truncation=True,
    max_length=4096
).to(device)

# k = int(row_dict["choices_len"])
input_len = inputs["input_ids"].shape[1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
        output_scores=True,
    )

In [132]:
generated_ids = outputs.sequences[0][input_len:]
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

In [133]:
print(generated_text)

 중소기업의 글로벌 판로 확대를 위해 2020년 10월 1일부터 2021년 1월 31일까지 쇼피 플랫폼에 입점한 중소기업에게 100만 마이린(약 10만 원)의 할인 쿠폰을 제공한다. 또한, 쇼피는 쇼피 플랫폼에 입점한 중소기업의 제품을 동남아 시장에 효과적으로 홍보하기 위해 쇼피의 마케팅 툴을 지원한다. 이와 함께 서울산업진흥원은 쇼피 플랫폼에 입점한 중소기업의 제품을 동남아 시장에 효과적으로 홍보하기 위해 쇼피의 마케팅 툴을 지원한다. 이와 함께 서울산업진흥원은 쇼피 플랫폼에 입점한 중소기업의 제품을 동남아 시장에 효과적으로 홍보하기 위해 쇼피의 마케groundColor

</think>

문제 분석 및 해결 전략:

1. **문제 정의**:
   - 중국의 푸둥자유무역지대에서 외국은행의 설립 자유화 조치가 발표됨에 따라, 이 조치가 중국의 금융시장 개혁과 국제금융센터로서의 지위 강화에 어떤 영향을 미칠 수 있는지 분석해야 한다.
   - 또한, 이 조치가 홍콩의 역외 위안화금융센터 지위에 미칠 영향과, 국내외 금융기관 간 경쟁 구도 변화를 고려해야 한다.

2. **문제 분석**:
   - **외국은행의 자유화 조치**:
     - 외국은행이 100% 자회사 또는 합작법인을 설립할 수 있도록 허용됨.
     - 이전에는 사무소 설립 후 2년 후 법인 전환 신청이 필요했으며, 심사를 거쳐 1년에 1개 지점만 설립 가능.
     - 이 조치는 외국은행의 중국 시장 진입 장벽을 낮추고, 글로벌 은행들의 상하이 진출 경쟁을 촉진할 것으로 예상됨.
   - **홍콩의 역외 위안화금융센터**:
     - 홍콩은 역외 위안화 예금 1조위안 중 약 7000억위안을 보유하고 있으며, 이는 위안화 국제화의 핵심 요소.
     - 그러나 상하이의 금융 개혁 조치가 시행되면, 홍콩으로 몰렸던 역외 금융 및 위안화 결제가 상하이로 이전될 가능성이 높음.
   - **중국의 금융개혁 전략**:
     - 외국은행의 자유로운 시장 진입을 통해 국제금융센터로서의 지위를 강화

---

In [105]:
valid_gen_df[valid_gen_df['is_correct'] == False].head()

,id,choices_len,answer,predicted_answer,is_correct,generated_text,is_retry,top5_candidates,digit_probs_1_to_k,digit_margin_top1_minus_top2,digit_top1,digit_top2,prompt
10,generation-for-nlp-2392,5,1,2,False,<think>\n\n</think>\n\n2,False,"[{'rank': 1, 'token_id': 17, 'token': '2', 'lo...","[0.4802214801311493, 0.49546536803245544, 0.01...",0.015244,2,1,<|im_start|>system\n당신은 논리적인 **텍스트 분석 및 독해 전문가...
11,generation-for-nlp-552,4,4,2,False,<think>\n\n</think>\n\n2,False,"[{'rank': 1, 'token_id': 17, 'token': '2', 'lo...","[0.0797504335641861, 0.5040382742881775, 0.274...",0.229997,2,3,<|im_start|>system\nYou are a student solving ...
17,generation-for-nlp-761,4,1,4,False,<think>\n\n</think>\n\n4,False,"[{'rank': 1, 'token_id': 19, 'token': '4', 'lo...","[0.17649178206920624, 0.009956984780728817, 0....",0.626946,4,1,<|im_start|>system\nYou are a student solving ...
28,generation-for-nlp-679,4,2,1,False,<think>\n\n</think>\n\n1,False,"[{'rank': 1, 'token_id': 16, 'token': '1', 'lo...","[0.4240732491016388, 0.24543477594852448, 0.06...",0.162809,1,4,<|im_start|>system\nYou are a student solving ...
37,generation-for-nlp-689,4,3,4,False,<think>\n\n</think>\n\n4,False,"[{'rank': 1, 'token_id': 19, 'token': '4', 'lo...","[0.04952811822295189, 0.02339542657136917, 0.0...",0.856247,4,1,<|im_start|>system\nYou are a student solving ...


In [69]:
def process_dataset(
    df: pd.DataFrame,
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str,
    max_new_tokens: int,
    desc: str = "Processing",
    threshold=0.66,
) -> pd.DataFrame:
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        row_dict = row.to_dict()
        result = generate_for_row_with_top5(
            row_dict=row_dict,
            builder=builder,
            tokenizer=tokenizer,
            model=model,
            device=device,
            max_new_tokens=max_new_tokens,
        )
        if result['digit_probs_1_to_k'][int(result['predicted_answer']) - 1] < threshold:
            print("애매한 답변으로 재생성...")
            result = generate_for_row_with_retry(
            row_dict=row_dict,
            builder=builder,
            tokenizer=tokenizer,
            model=model,
            device=device,
            generated_text=result['generated_text'],
            max_new_tokens=max_new_tokens,
        )
            

        results.append(result)

    return pd.DataFrame(results)

In [65]:
# 1. 틀린 문제만 필터링
failed_df = valid_gen_df[valid_gen_df['is_correct'] == False].copy()

# 2. 리스트 내 float 숫자를 소수점 둘째 자리까지 문자열로 변환하는 함수
def format_probs(prob_list):
    if isinstance(prob_list, list):
        return [round(p, 2) for p in prob_list]
    return prob_list

# 3. 데이터 가공
# 시각적 가독성을 위해 확률 리스트를 포맷팅합니다.
failed_df['formatted_probs'] = failed_df['digit_probs_1_to_k'].apply(format_probs)

# 4. 보기 좋게 선택된 컬럼만 출력 (Style 적용)
styled_df = failed_df[['id', 'answer', 'predicted_answer', 'formatted_probs']].style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
}).set_table_styles([
    dict(selector='th', props=[('text-align', 'center'), ('background-color', '#f4f4f4')])
])

In [66]:
styled_df

,id,answer,predicted_answer,formatted_probs
10,generation-for-nlp-2392,1,2,"[0.11, 0.87, 0.01, 0.01, 0.0]"
11,generation-for-nlp-552,4,2,"[0.02, 0.81, 0.1, 0.06]"
17,generation-for-nlp-761,1,4,"[0.18, 0.01, 0.01, 0.8]"
28,generation-for-nlp-679,2,1,"[0.53, 0.22, 0.05, 0.21]"
37,generation-for-nlp-689,3,4,"[0.05, 0.02, 0.02, 0.91]"
39,generation-for-nlp-1103,1,4,"[0.01, 0.0, 0.0, 0.98]"
43,generation-for-nlp-1096,3,1,"[0.72, 0.16, 0.06, 0.06]"
59,generation-for-nlp-973,1,2,"[0.21, 0.68, 0.05, 0.06]"
71,generation-for-nlp-578,3,1,"[0.68, 0.05, 0.12, 0.15]"
74,generation-for-nlp-721,1,4,"[0.02, 0.0, 0.0, 0.98]"


In [43]:
def check_answer_rank(row):
    # 1. 확률 리스트와 실제 정답 가져오기
    probs = row['digit_probs_1_to_k']
    ans = row['answer'] # 정답이 숫자(1, 2, 3...) 형태라고 가정
    
    try:
        # 정답 인덱스 (리스트는 0부터 시작하므로 -1)
        target_idx = int(ans) - 1
        target_prob = probs[target_idx]
        
        # 2. 내림차순 정렬하여 순위 확인
        sorted_probs = sorted(probs, reverse=True)
        rank = sorted_probs.index(target_prob) + 1 # 1위, 2위...
        
        return rank
    except:
        return None

# 틀린 문제들에 대해 정답의 확률 순위 계산
failed_df['answer_rank'] = failed_df.apply(check_answer_rank, axis=1)

# 결과 집계
second_rank_count = len(failed_df[failed_df['answer_rank'] == 2])
total_failed = len(failed_df)
ratio = (second_rank_count / total_failed) * 100

print(f"📊 오답 분석 결과")
print(f"- 전체 오답 개수: {total_failed}개")
print(f"- 정답 확률이 2순위였던 오답: {second_rank_count}개")
print(f"- 오답 중 '아까운' 문제 비중: {ratio:.2f}%")

# 실제 데이터 확인 (정답이 2순위인 것들만 추출)
top_near_misses = failed_df[failed_df['answer_rank'] == 2][['id', 'answer', 'predicted_answer', 'digit_probs_1_to_k']]
top_near_misses.head(13)

📊 오답 분석 결과
- 전체 오답 개수: 20개
- 정답 확률이 2순위였던 오답: 13개
- 오답 중 '아까운' 문제 비중: 65.00%


,id,answer,predicted_answer,digit_probs_1_to_k
10,generation-for-nlp-2392,1,2,"[0.4802214801311493, 0.49546536803245544, 0.01..."
17,generation-for-nlp-761,1,4,"[0.17649178206920624, 0.009956984780728817, 0...."
39,generation-for-nlp-1103,1,4,"[0.011951402761042118, 0.00490484619513154, 0...."
59,generation-for-nlp-973,1,2,"[0.21127209067344666, 0.6819946765899658, 0.04..."
74,generation-for-nlp-721,1,4,"[0.015083985403180122, 0.0042546335607767105, ..."
99,generation-for-nlp-2140,4,1,"[0.8210901021957397, 0.02403191849589348, 0.01..."
104,generation-for-nlp-885,3,1,"[0.5448269844055176, 0.062090519815683365, 0.3..."
106,generation-for-nlp-826,1,3,"[0.2252793312072754, 0.14545126259326935, 0.46..."
127,generation-for-nlp-675,2,1,"[0.9945474863052368, 0.004001474007964134, 0.0..."
143,generation-for-nlp-530,4,3,"[0.02465880662202835, 0.015191849321126938, 0...."


In [47]:
# 틀린 예측의 확률 값의 평균
pred_probs = failed_df.apply(lambda x: round(x['digit_probs_1_to_k'][int(x['predicted_answer'])-1], 4), axis=1)
print(pred_probs.mean())

0.6612199999999999


In [72]:
import pandas as pd

# 1. top_near_misses와 valid_df를 ID 기준으로 병합
# valid_df에 'id'와 'paragraph' 컬럼이 있다고 가정합니다.
near_miss_with_text = pd.merge(
    top_near_misses, 
    valid_df[['id', 'paragraph']], 
    on='id', 
    how='left'
)

# 2. 가독성을 위해 출력 설정 (텍스트가 길 수 있으므로 왼쪽 정렬 및 줄바꿈 허용)
pd.set_option('display.max_colwidth', None) # 텍스트 생략 방지

# 3. 주요 정보 위주로 출력
# id, 정답, 예측값, 확률리스트, 그리고 원본 지문 순서
# 1. 리스트의 각 요소를 소수점 둘째 자리까지 반올림하는 함수 적용
near_miss_with_text['digit_probs_1_to_k'] = near_miss_with_text['digit_probs_1_to_k'].apply(
    lambda x: [round(prob, 2) for prob in x] if isinstance(x, list) else x
)

# 2. 출력 설정 유지
pd.set_option('display.max_colwidth', None)

# 3. 데이터 출력
display(near_miss_with_text[['id', 'answer', 'predicted_answer', 'digit_probs_1_to_k', 'paragraph']])


,id,answer,predicted_answer,digit_probs_1_to_k,paragraph
0,generation-for-nlp-2392,1,2,"[0.48, 0.5, 0.01, 0.01, 0.0]","대법관추천위원회가 다음달 임기를 마치는 민일영 대법관 후임으로 강형주 법원행정처 차장(56·사법연수원 13기), 성낙송 수원지방법원장(57·14기), 이기택 서울서부지방법원장(56·14기) 등 3명을 추천했다. 양승태 대법원장은 이들 중 한 명을 선정해 이르면 이번주에 대통령에게 임명을 제청할 예정이다. 대법원 구성 다양화를 요구해 온 재야 법조계는 “이번에도 ‘50대, 남성, 법관 출신’이라는 획일적인 틀을 벗어나지 못했다”고 비판했다.추천위는 4일 서울 서초동 대법원 청사에서 회의를 열어 법원 안팎에서 천거된 대상자들을 최종 심사한 뒤 이같이 결정했다. 추천받은 후보자들은 모두 서울대 법대 출신의 현직 법관으로 채워졌다.후보자들이 주력해온 분야와 업적 등은 조금씩 다르다. 강 차장은 전남 함평에서 태어나 광주제일고를 졸업했다. 서울중앙지법 근무 당시 영장전담과 형사합의부 재판장 등을 지낸 대표적인 형사 전문가다. 서울고법 근무 때는 민청학련 사건에 연루됐던 최권행 서울대 불어불문학과 교수와 제정구 전 국회의원 등의 재심사건에서 무죄를 선고했다.성 원장은 경남 산청에서 태어나 경기고를 졸업했다. 법원행정처 사법정책심의관과 공보관, 서울고법 부장판사 등을 지냈다. 양형위원회 초대 상임위원으로 양형 기준 기초를 마련했으며 서울중앙지법에서 일할 때는 성폭력 피해자 증인지원 프로그램을 처음 도입했다.이 원장은 서울 출신이며 경성고를 졸업했다. 대법원 재판연구관과 특허법원 부장판사, 서울고법 부장판사 등을 거쳤다. 법원 내 대표적인 민법 전문가로 손꼽히며 지식재산권법 연구회장을 지냈다.김종인 추천위원장은 “추천을 받은 후보자들은 법률가로서의 자질이 뛰어나고 풍부한 경륜과 인품, 도덕성과 청렴성까지 두루 갖췄다”고 평가했다. 대한변협이 강 변호사와 함께 공개 추천한 김선수 변호사(17기)는 대한변협 외 다른 추천인이 있다는 이유로 심사 대상에는 포함됐으나 최종 선정되지는 못했다.대한변협은 “대법원이 말해온 구성의 다양화가 헛구호였음이 여실히 드러났다”고 비판했다. 대한변협은 이날 긴급 성명을 내고 “최근 대법원 전원합의체가 반대의견 하나 없이 전원일치 판결을 잇달아 선고하는 건 구성의 다양화에 실패했기 때문”이라며 “이번 후보 추천에서도 법관 순혈주의를 고수했다”고 지적했다. 추천위 관계자는 “심사 대상자 중 외부인사가 5명에 불과했고 이들 가운데선 대법관으로서의 자질 등 자격요건을 모두 갖춘 후보를 찾기 어렵다고 판단했다”고 해명했다."
1,generation-for-nlp-761,1,4,"[0.18, 0.01, 0.01, 0.8]",I. 다른 직장을 구하기 위해 직장을 그만둔 메리. II. 45세에 직장에서 은퇴하여 자신의 꿈을 이룬 존. III. 파트타임으로 일하고 있지만 풀타임으로 일하고 싶은 다이앤.
2,generation-for-nlp-1103,1,4,"[0.01, 0.0, 0.0, 0.98]","이제 우리는 사회를 조직했고, ""모든 사람은 왕""이라는 모토를 가진 사회를 ""부를 공유하는 사회""라고 부릅니다. 우리는 국내 거물의 부를 제한할 것을 제안합니다. 미국 모든 가족은 평균 15,000달러의 부가 있습니다. 오늘 현재의 상황입니다. 우리는 균등하게 나누는 것을 제안하지는 않습니다. 부의 분배를 제안하지는 않지만, 모든 가족에게 가해지는 빈곤을 제한할 것을 제안합니다. 평등을 보장하려고 노력하리라고는 말하지 않을 것입니다... 하지만 평균의 3분의 1은 한 가족에게는 너무 낮으며, 일가족에 약 5,000달러의 부를 보장해야 한다고 주장합니다. 이는 주택, 자동차, 라디오, 일상 편의시설, 자녀를 교육할 기회로는 충분합니다.… 우리는 재산을 제한할 것입니다. 현재 계획은 어떤 사람도 $50,000,000 초과 소유를 허용하지 않는 겁니다. 그 한도 내에서 프로그램의 균형을 이룰 수 있다고 생각합니다. —루이지애나주 상원의원 휴이 P. 롱, 라디오 연설, 1934년 2월 23일 롱 상원의원의 ""부를 공유하는 사회""는 1934년에 많은 추종자를 끌어모았다."
3,generation-for-nlp-973,1,2,"[0.21, 0.68, 0.05, 0.06]","정서적 애착에 대한 할로우(Harlow)의 연구에서 새끼 원숭이들을 우리에 넣고 ""철사로 만든"" 엄마와 ""헝겊으로 만든"" 엄마를 모두 제공했습니다. 그런 다음 연구자들은 원숭이들이 두 ""엄마"" 중 하나에게 애착을 형성하는지 확인하기 위해 다양한 자극의 도입과 함께 우유병을 한 어미에게서 다른 어미로 옮겼습니다."
4,generation-for-nlp-721,1,4,"[0.02, 0.0, 0.0, 0.98]",달러화 시장 달러의 가치
5,generation-for-nlp-2140,4,1,"[0.82, 0.02, 0.02, 0.1, 0.04]","한민구 국방부 장관(사진)은 10일 인천지역 모 육군부대 A사단장(소장)이 여군 부사관을 성추행한 혐의에 대해 “최근 일련의 군 기강 해이 사건들은 군의 명예를 떨어뜨리고 국민의 신뢰를 저버리는 행위로 철저한 반성이 필요하다”고 말했다.한 장관은 이날 국방부 청사에서 긴급 주요지휘관 화상회의를 열고 고위 장성의 잇단 일탈행위와 각종 병영 내 사건·사고 등 군 기강 문란을 강하게 질책했다. 한 장관은 성군기 위반행위와 군사기밀 유출, 군납 및 방산비리 사례를 일일이 열거하면서 재발 방지책을 마련하라고 주문했다.회의에 참석한 국방부 고위 관계자는 이날 한 장관이 “군 기강을 저해한 사람은 지위고하를 막론하고 반드시 일벌백계할 것이라는 방침을 강조했다”고 말했다.군사법원은 이날 A사단장에게 ‘군인 등 강제추행죄’를 적용해 구속영장을 발부했다. A사단장은 사단 예하 다른 부대에서 상사로부터 성추행 피해를 입고 사단사령부로 전출된 피해자를 집무실에서 껴안는 등 성추행한 것으로 알려져 지난 9일 긴급체포됐다."
6,generation-for-nlp-885,3,1,"[0.54, 0.06, 0.34, 0.06]","초기 읽기 지도에 대한 ‘발음 중심 교수법’(code-based phonic approach)에서 1학년 학생들은 문자 b, a, s, g의 소리를 배웁니다. 이러한 접근법의 기반이 되는 이론에 따르면,"
7,generation-for-nlp-826,1,3,"[0.23, 0.15, 0.47, 0.16]",Bessie는 1/4티스푼의 물을 0.5갤런의 물에 섞은 물병에서 물 한 모금을 미셨을 때 단맛을 거의 느낄 수 없었습니다.
8,generation-for-nlp-675,2,1,"[0.99, 0.0, 0.0, 0.0]","선거인단 제도의 구조를 감안할 때, 대선 후보들은 주로 어떤 경향을 갖습니까?"
9,generation-for-nlp-530,4,3,"[0.02, 0.02, 0.84, 0.12]","버킹엄 궁전, 1839년 5월 10일. 여왕은 지난 나흘 동안 자신이 해야 한 많은 일을 고려하면 자신이 화요일에 글로스터 가문에서 열리는 파티, 수요일에 열리는 에인션트 콘서트, 목요일에 열리는 노섬버랜드 가문의 무도회에 가면 피로를 느낄까봐 두렵다는 내용으로 케임브리지 공작에게 편지를 쓰는 것이 해로울 것이라고 생각하는지 멜버른 경에게 묻는 것을 잊어버렸다. 여왕이 수요일 에인션트 콘서트에 갔다면 월요일 이곳에서 자신의 콘서트를 제하고라도 나흘 밤의 피로가 누적되어 정말 여왕처럼 지쳤을 것이다. 그러나 멜버른 경이 에인션트 콘서트에는 영국 가수들만 있기에 가야 한다고 생각한다면, 여왕은 공연 하나를 보기 위해 갈 수 있다. 그러나 지금은 피곤한 시기라서 가능하면 피하고 싶어한다.… 보수당과의 협상이 거의 끝났고 멜버른 경이 여기 왔으니, 여왕은 멜버른 경이 일요일 함께 식사하는 데 반대하지 않기를 바라겠는가? 빅토리아 여왕의 편지, 3권 중 1권, 1837년~1843년: 1837년과 1861년 사이의 여왕 폐하의 서신에서 발췌."


In [78]:
# 1. 'question_plus' 값이 있는(NaN이 아닌) 행들만 필터링하여 id 리스트 생성
ids_with_question_plus = test_df[test_df['question_plus'].notna()]['id'].tolist()

# 2. 개수 확인 및 일부 출력
print(f"✅ 'question_plus'가 포함된 ID 개수: {len(ids_with_question_plus)}개")
print(f"📋 상위 10개 ID: {ids_with_question_plus}")

✅ 'question_plus'가 포함된 ID 개수: 44개
📋 상위 10개 ID: ['generation-for-nlp-6', 'generation-for-nlp-10', 'generation-for-nlp-15', 'generation-for-nlp-18', 'generation-for-nlp-20', 'generation-for-nlp-24', 'generation-for-nlp-28', 'generation-for-nlp-30', 'generation-for-nlp-32', 'generation-for-nlp-54', 'generation-for-nlp-55', 'generation-for-nlp-60', 'generation-for-nlp-65', 'generation-for-nlp-68', 'generation-for-nlp-72', 'generation-for-nlp-77', 'generation-for-nlp-78', 'generation-for-nlp-83', 'generation-for-nlp-111', 'generation-for-nlp-115', 'generation-for-nlp-121', 'generation-for-nlp-123', 'generation-for-nlp-130', 'generation-for-nlp-134', 'generation-for-nlp-142', 'generation-for-nlp-170', 'generation-for-nlp-176', 'generation-for-nlp-182', 'generation-for-nlp-187', 'generation-for-nlp-189', 'generation-for-nlp-193', 'generation-for-nlp-197', 'generation-for-nlp-200', 'generation-for-nlp-203', 'generation-for-nlp-209', 'generation-for-nlp-364', 'generation-for-nlp-369', 'generati